<a href="https://colab.research.google.com/github/achiral/styleGAN2_pytorch/blob/main/01_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2

# GPUガチャ  
Tesla V100 > Tesla P100 > Tesla T4

In [ ]:
# GPU情報の確認
!nvidia-smi

Sat Jun 19 16:12:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Copy dataset from Google drive  
### datasetファイルの展開に少し時間がかかります。  
### datasetが完全に展開されるまで待ってから学習を開始すると、「学習ファイルがありません」というエラーを回避できます。

In [ ]:
# google driveからcolaboにファイルをコピーするメソッド
import requests

def download_file_from_google_drive(id, destination):
    print('---- download_file_from_google_drive start ----')
    URL = "https://docs.google.com/uc?export=download"

    try:
        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        print('response.status_code: {}'.format(response.status_code))
        print('response.headers: {}'.format(response.headers))

        token = get_confirm_token(response)
        print('token'.format(token))

        if token:
            print(token)
            params = { 'id' : id, 'confirm' : token }
            response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)
    except Exception as e:
        print(e)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        print('key: {}'.format(key))
        print('value: {}'.format(value))
       
        if key.startswith('download_warning'):
            return value
            
        if key == 'NID':
            print('NID')
            token = value.split('=')
            print(token)
            return token[1]

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
# 実行環境のrootディレクトリの確認
%cd /content/
!ls -lah

In [ ]:
# ソースコードのダウンロード
# %%time
# %cd /content/
# !rm -fr gan_sample
# !git clone https://github.com/ayukat1016/gan_sample.git

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# マウントしたGoogle Driveの情報
!ls gdrive -lah

In [ ]:
# datasetを配置するディレクトリを作成
!mkdir ./dataset
!mkdir /content/output

In [ ]:
# endless_summer datasetのダウンロード
# ★★★★ 7GBあるのでダウンロードに時間がかかる ★★★★
# https://drive.google.com/file/d/1LM4FtUltzS45PuFyfuSp3I8QdTD8Cu0F/view?usp=sharing
# file_id = '1LM4FtUltzS45PuFyfuSp3I8QdTD8Cu0F'
# destination = './dataset/endless_summer.zip'
# download_file_from_google_drive(file_id, destination)

In [ ]:
# %cd /content/gdrive/MyDrive/'Colab Notebooks'/lightweight-gan/results
# # フォルダを zip 圧縮する
# !zip -r ./inner-generated-100.zip ./inner-generated-100


# # 圧縮した zip ファイルをダウンロードする
# # from google.colab import files
# # files.download("./inner-generated-100.zip")

# %cd /content
# %pwd

In [ ]:
# dataset(zipでgdriveに格納)をColabにコピー
# datasetのサイズによって時間がかかる
%%time
# !cp "./gdrive/MyDrive/dataset/endless_summer.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/amime_128_mini.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner2_512.zip" "./dataset"
!cp "./gdrive/MyDrive/dataset/inner3_512.zip" "./dataset"
!cp "./gdrive/MyDrive/dataset/inner3_2048.zip" "./dataset"
!cp "./gdrive/MyDrive/dataset/inner3_2048alpha.zip" "./dataset"
!cp "./gdrive/MyDrive/dataset/skin_512.zip" "./dataset"
!cp "./gdrive/MyDrive/dataset/onepiece_512.zip" "./dataset"
# !cp "/content/gdrive/MyDrive/Colab Notebooks/lightweight-gan/results/inner-generated-100.zip" "./dataset"

In [ ]:
# コピー(またはダウンロード)したdataset解凍
%%time
# !unzip ./dataset/endless_summer.zip -d ./dataset
# !unzip ./dataset/amime_128_mini.zip -d ./dataset
# !unzip ./dataset/inner_512.zip -d ./dataset
# !unzip ./dataset/inner2_512.zip -d ./dataset
!unzip ./dataset/inner3_512.zip -d ./dataset
!unzip ./dataset/inner3_2048.zip -d ./dataset
!unzip ./dataset/inner3_2048alpha.zip -d ./dataset
!unzip ./dataset/skin_512.zip -d ./dataset
!unzip ./dataset/onepiece_512.zip -d ./dataset
# !unzip ./dataset/inner-generated-100.zip -d ./dataset

# データ件数の確認
!find dataset/* -type f | wc -l

In [ ]:
# 作業ディレクトリの移動
# %cd /content/gdrive/MyDrive/CV_Hands-on/06_Day3-2/02_StyleGAN
%cd /content/gdrive/MyDrive/Colab Notebooks/gan_sample/chapter7
%pwd
%ls -al

 # Training

In [ ]:
# 学習
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# ★★★★★　長時間の学習を行うとGoogle Driveの容量が足りなくなる事があるので注意!!!!!!　★★★★★
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# Google Driveの容量が少ない方は事前に容量を購入するか、下記のsave intervalの値を大きくしてください。
# --save_model_interval=32 \
# --save_metrics_interval=2 \
# --save_images_tensorboard_interval=8 \
# --save_images_interval=32 \
# 特に--model_pathに保存されるモデルファイルは1ファイルで300MBと大きいので、古いモデルファイルは消すようにしてください。

# それっぽい画像が生成されるまで、2〜3日、精度の良い画像が生成されるには1週間以上かかるかもしれません。
# 学習は--model_pathオプションに格納されている一番新しいモデルファイルを使用して継続学習を行います。
# --model_pathで指定したディレクトリにモデルファイルがない場合ははじめから学習を行います。
# fidスコア算出のためにinceptionの重みファイルを初回にダウンロードするので、学習が開始するまで時間(5分〜10分)がかかります。

# --model_pathに学習したモデルの保存先を指定します。
# --data_pathに学習するdata setのディレクトリを指定します。
# --resultsに学習した結果を保存するディレクトリを指定します。
# --cache_pathに一時ファイルの格納先を指定します。
# --tensorboad_pathにTensoboardのログファイルを保存するディレクトリを指定します。
# --save_model_interval: モデルを保存する間隔を指定します。32を指定すると32回のループ回数毎にモデルを保存します。
# --save_metrics_interval: 指標をTensorboardに出力する間隔を指定します。
# --save_images_tensorboard_interval: 生成した画像をTensorboadに出力する間隔を指定します。
# --save_images_interval: 生成した画像をファイルとして保存する間隔を指定します。
# --generator_train_num: generatorが連続して学習する回数を指定します。とりあえず4で良いと思います。
# --discriminator_train_num: discriminatorが連続して学習する回数を指定します。とりあえず4で良いと思います。
# --reverse_decay: 1のままで
# --g_reg_interval: generatorの正則処理を行う間隔を指定します。とりあえず4で良いと思います。
# --d_reg_interval: discriminatorの正則処理を行う間隔を指定します。とりあえず16で良いと思います。
# --fid_score_interval: fidスコアを算出する間隔2048をしてします。fidのスコアの算出は非常に時間がかかるので、実行したくない場合は大きな値32000を指定してください。

  !python ./stylegan2_pytorch/training.py \
--batch_size=                           4 \
--resolution=                        2048 \
--max_loop_num=                     32000 \
--model_path=              ./models/inner \
--data_path= /content/dataset/inner3_2048 \
--results=                ./results/inner \
--cache_path=                     ./cache \
--tensorboard_path=                ./logs \
--save_model_interval=                128 \
--save_metrics_interval=            32000 \
--save_images_tensorboard_interval= 32000 \
--save_images_interval=               128 \
--generator_train_num=                  4 \
--discriminator_train_num=              4 \
--reverse_decay=                        1 \
--g_reg_interval=                       4 \
--d_reg_interval=                      16 \
--fid_score_interval=               32000